In [1]:
import re
import tweepy
from tweepy import OAuthHandler
import numpy as np
from textblob import TextBlob
import datetime
import time


In [2]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # set twitter api credentials
        consumer_key= 'MOe623rxcqck6x8y5XhzK8MJT'
        consumer_secret= 'mcBq9Km1f3OYERRD6vKmOfWSgCjsqzXAreIsn8klxAtPIo40E7'
        access_token='913787859630460928-RXF8NVN3gGbxD64NCZ7wBma5M2WPwlv'
        access_token_secret='P4UU9I2DimdnUown2EM6p4WZ0ftdPNsysDNW6xGh0Ts4f'

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        return analysis.sentiment.polarity
    
    def get_polarity(self, query, count = 100):
        # run polarity analysis on tweets
        polarity = 0
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
            for tweet in fetched_tweets:
                polarity += self.get_tweet_sentiment(tweet.text) * 1.0 / count
            return polarity
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

    def get_tweets(self, query, count = 100):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        results = {}
        positive = 0
        negative = 0
        ptweet = ""
        ntweet = ""

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                sentiment = self.get_tweet_sentiment(tweet.text)
                # saving sentiment of tweet
                if(sentiment > 0):
                    positive += 1
                    if positive == 1:
                        ptweet = tweet.text
                elif(sentiment < 0):
                    negative += 1
                    if negative == 1:
                        ntweet = tweet.text

            results["positive_tweet"] = ptweet
            results["negative_tweet"] = ntweet
            results["positive_percentage"] = positive / 100.0
            results["negative_percentage"] = negative / 100.0
            results["neutral_percentage"] = (100 - positive - negative) / 100.0
            # return a posive tweet, a negative tweet, the positive tweet percent and the negative tweet percent 
            return results
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [3]:
api = TwitterClient()
# calling function to get tweets
tweets = api.get_tweets(query = 'bitcoin, price, crypto')

# printing a positive tweet
print "\n\nPositive tweets: \n%s" % tweets["positive_tweet"]

# printing a negative tweet
print "\n\nNegative tweets: \n%s\n" % tweets["negative_tweet"]

# print the positve percentage
print "Positive tweets percentage: {}%".format(100*tweets["positive_percentage"])

# print the negative tweets percentage
print "Negative tweets percentage: {}%".format(100*tweets["negative_percentage"])

# print the neutral tweets percentage
print "Neutral tweets percentage: {}%".format(100*tweets["neutral_percentage"])



Positive tweets: 
One person is probably responsible for almost 600 percent of Bitcoin's price rise https://t.co/lCjb6lKlfy

FIVE AMA… https://t.co/d5jdKO1STy


Negative tweets: 
ALERT: All cryptos going down at least 85% because of South Korea, China, Japan, Russia, USA and Europe ALL BANNING… https://t.co/wPYWp2xzN2

Positive tweets percentage: 21.0%
Negative tweets percentage: 49.0%
Neutral tweets percentage: 30.0%


In [4]:
while True:
    # fetch tweets by keywords
    polarity = api.get_polarity(query=['bitcoin, price, crypto'], count=100)
    print polarity
    time.sleep(60)

-0.156738305676


KeyboardInterrupt: 